In [1]:
import os
os.chdir("../")
import pickle
import pandas as pd

from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
from typing import Callable
from src.config import Config
from src.dataset import Interactions
from src.metrics import PrecisionAtK, RecallAtK, FScoreAtK
from src.optimizer import OptunaMaximizer

In [2]:
with Config.PREPROCESSED_INTERACTIONS_PATH.open("rb") as file_object:
    interactions = Interactions(pickle.load(file_object), test_days=7, valid_days=14)

In [3]:
interactions.train = interactions.train[["user_id", "item_id", "interest_score"]]
interactions.test = interactions.test[["user_id", "item_id", "interest_score"]]
interactions.valid = interactions.valid[["user_id", "item_id", "interest_score"]]

In [4]:
interactions.train

,user_id,item_id,interest_score
0,126706,14433,0.400
1,127290,140952,0.290
2,66991,198453,0.445
3,46791,83486,0.615
4,79313,188770,0.940
...,...,...,...
1532993,153908,98585,0.220
1532994,154008,251969,0.020
1532995,154892,298192,0.840
1532996,156948,38118,0.890


In [5]:
interactions.test

,user_id,item_id,interest_score
1517915,101642,319500,0.835
1517917,130425,193445,0.490
1517918,93986,80733,0.235
1517919,159466,124115,0.420
1517920,158775,223806,0.270
...,...,...,...
1530838,141930,219928,0.450
1530839,53358,42887,0.290
1530840,151170,284652,0.135
1530841,141293,273421,0.240


In [6]:
interactions.valid

,user_id,item_id,interest_score
1503047,22032,287219,0.275
1503048,84214,121609,1.000
1503049,28992,11482,0.110
1503050,23345,281921,1.000
1503051,49466,2880,0.010
...,...,...,...
1517908,81768,294370,0.850
1517910,158991,99669,0.815
1517911,77232,142149,0.020
1517912,17843,174535,0.060


In [7]:
csr_train = csr_matrix(
    (interactions.train["interest_score"], 
    (interactions.train["user_id"], interactions.train["item_id"]))
)
csr_train

<159613x321752 sparse matrix of type '<class 'numpy.float32'>'
	with 1306588 stored elements in Compressed Sparse Row format>

In [8]:
class Objective:
    def __init__(self, train: csr_matrix, valid: pd.DataFrame, metric: Callable):
        self.train = train
        self.valid = valid
        self.metric = metric

    def __call__(self, trial) -> float:
        search_space = {
            "factors": trial.suggest_int("factors", 4, 256),
            "regularization": trial.suggest_float("regularization", 1e-8, 0.1),
            "iterations": trial.suggest_int("iterations", 8, 64),
            "num_threads": Config.NUM_THREADS
        }
        model = AlternatingLeastSquares(**search_space)
        model.fit(self.train)
        predictions = pd.DataFrame(self.valid["user_id"].drop_duplicates())
        predictions["item_id"] = predictions["user_id"].apply(
            lambda user_id: 
                model.recommend(
                    user_id, 
                    self.train[user_id],
                    N=Config.K,
                    filter_already_liked_items=True
                )[0]
        )
        predictions = predictions.explode("item_id")
        return self.valid["user_id"].drop_duplicates().apply(
            lambda user_id: 
                self.metric.calculate(
                    predictions[predictions["user_id"] == user_id]["item_id"],
                    self.valid[self.valid["user_id"] == user_id]["item_id"]
                )
        ).mean()

In [9]:
recall = RecallAtK(Config.K)
precision = PrecisionAtK(Config.K)

In [10]:
precision_objective = Objective(csr_train, interactions.valid, precision)
precision_optimizer = OptunaMaximizer(precision_objective, "ALS precision maximize")
precision_best_trial = precision_optimizer.optimize(Config.ITERATIONS, Config.STOP_ITERATION)

[I 2023-03-11 14:25:03,901] A new study created in memory with name: ALS precision maximize
/home/aveitsme/miniconda3/envs/core/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

[I 2023-03-11 14:25:14,031] Trial 0 finished with value: 0.0017079332996462837 and parameters: {'factors': 68, 'regularization': 0.08287552288829421, 'iterations': 33}. Best is trial 0 with value: 0.0017079332996462837.


  0%|          | 0/19 [00:00<?, ?it/s]

...

[I 2023-03-11 14:36:00,750] Trial 33 finished with value: 0.002374936836786261 and parameters: {'factors': 239, 'regularization': 0.08169216574448732, 'iterations': 45}. Best is trial 15 with value: 0.0024759979787771676.


  0%|          | 0/37 [00:00<?, ?it/s]

[I 2023-03-11 14:36:19,097] Trial 34 finished with value: 0.002102071753410816 and parameters: {'factors': 173, 'regularization': 0.08244262301635853, 'iterations': 37}. Best is trial 15 with value: 0.0024759979787771676.


  0%|          | 0/40 [00:00<?, ?it/s]

[I 2023-03-11 14:36:35,583] Trial 35 finished with value: 0.0020515411824153637 and parameters: {'factors': 137, 'regularization': 0.0679144638072739, 'iterations': 40}. Best is trial 15 with value: 0.0024759979787771676.
EarlyStopping Exceeded: No new best scores on iters 20


In [11]:
recall_objective = Objective(csr_train, interactions.valid, recall)
recall_optimizer = OptunaMaximizer(recall_objective, "ALS recall maximize")
recall_best_trial = recall_optimizer.optimize(Config.ITERATIONS, Config.STOP_ITERATION)

[I 2023-03-11 14:36:35,664] A new study created in memory with name: ALS recall maximize
/home/aveitsme/miniconda3/envs/core/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

[I 2023-03-11 14:37:02,393] Trial 0 finished with value: 0.01037806531237557 and parameters: {'factors': 234, 'regularization': 0.032987659441796865, 'iterations': 56}. Best is trial 0 with value: 0.01037806531237557.


  0%|          | 0/31 [00:00<?, ?it/s]

...

[I 2023-03-11 14:51:50,061] Trial 52 finished with value: 0.01239436311821355 and parameters: {'factors': 244, 'regularization': 0.08741060089339941, 'iterations': 14}. Best is trial 34 with value: 0.013394277105747546.


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-03-11 14:52:06,039] Trial 53 finished with value: 0.011878734369891522 and parameters: {'factors': 220, 'regularization': 0.09340844808662657, 'iterations': 10}. Best is trial 34 with value: 0.013394277105747546.


  0%|          | 0/15 [00:00<?, ?it/s]

[I 2023-03-11 14:52:25,518] Trial 54 finished with value: 0.012604390956325399 and parameters: {'factors': 246, 'regularization': 0.0795730926831978, 'iterations': 15}. Best is trial 34 with value: 0.013394277105747546.
EarlyStopping Exceeded: No new best scores on iters 20


In [12]:
precision_model = AlternatingLeastSquares(**precision_best_trial.params)
precision_model.fit(csr_train)

  0%|          | 0/39 [00:00<?, ?it/s]

In [13]:
recall_model = AlternatingLeastSquares(**recall_best_trial.params)
recall_model.fit(csr_train)

  0%|          | 0/8 [00:00<?, ?it/s]

In [14]:
models = {
    "recall_model": recall_model,
    "precision_model": precision_model
}

In [15]:
metrics = {
    "precision": precision,
    "recall": recall
}

In [16]:
for model_name, model in models.items():
    pred = pd.DataFrame(interactions.test["user_id"].drop_duplicates())
    print(f"Model '{model_name}':")
    pred["item_id"] = pred["user_id"].apply(
        lambda user_id: 
            model.recommend(
                user_id, 
                csr_train[user_id],
                N=Config.K,
                filter_already_liked_items=True
            )[0]
    )
    pred = pred.explode("item_id")
    for metric_name, metric in metrics.items():
        metric_val = interactions.test["user_id"].drop_duplicates().apply(
            lambda user_id: 
                metric.calculate(
                    pred[pred["user_id"] == user_id]["item_id"],
                    interactions.test[interactions.test["user_id"] == user_id]["item_id"]
                )
        ).mean()
        print(f"\tMetric {metric_name}@{Config.K}: {metric_val}")

Model 'recall_model':
	Metric precision@10: 0.0018301528847226843
	Metric recall@10: 0.00969147252207179
Model 'precision_model':
	Metric precision@10: 0.0015734851021091366
	Metric recall@10: 0.008407195601705145


In [19]:
with (Config.MODEL_PATH / "als_trial.pickle").open("wb") as file_object:
    pickle.dump(recall_best_trial, file_object)